**Michael Bocamazo**

**Question 2**: How well does logistic regression perform as a baseline with minimal feature engineering and one-hot expansion?

**Date**: 2016/10/24

**Methods**: Select down to only features usable within logistic regression.  Choose a few appropriate for a OHE.  Train logistic regression.  Cross-validate the ridge term.  Add in OHE feats.  Examine resultant weights, and try to identify where prediction fails.

**Conclusion**: 

In [1]:
import os, sys
import csv
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import seaborn as sns
import copy
%matplotlib inline
sns.set_style("darkgrid", {"grid.linewidth": .5, "axes.facecolor": ".9"})

In [2]:
import ML_utils as ml_ut

From the previous question, we have a clean data frame, and a data frame with expanded features.

In [3]:
df_c = pd.read_csv('SM_clean.csv')

In [4]:
df_c.columns

Index(['SURVEY', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE',
       'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'CAR_AV', 'TRAIN_TT',
       'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS', 'CAR_TT',
       'CAR_CO', 'CHOICE'],
      dtype='object')

In [7]:
df_e = pd.read_csv('SM_expand.csv')

In [8]:
df_e.columns

Index(['SURVEY', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE',
       'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'CAR_AV', 'TRAIN_TT',
       'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS', 'CAR_TT',
       'CAR_CO', 'CHOICE', 'min_CO', 'ratio_TRAIN_CO', 'ratio_SM_CO',
       'ratio_CAR_CO', 'min_TT', 'ratio_TRAIN_TT', 'ratio_SM_TT',
       'ratio_CAR_TT'],
      dtype='object')

### Features to include
We can make a simple list of features that would be appropriate for a logistic regression model without any transformations.  The certain ones are the numerics and binary features.  Less certain are the ordinal encoded features.  The simply encoded features require an OHE.

The **pure feats** are the numeric features that we expect to get a straight correlation to choice, except for cost, which needs an adjustment based on GA to work well.  

The **simple feats** are binary or ordinal encoded features.  The INCOME and AGE features both have a category for unknowns, at the endpoint.  If the latent classes behind the unknowns are equally distributed among the values, this acts as a regularizer.  In both of these features they occupy the greatest value.  They could be made to occupy the the mid value for a cleaner regularization.

The **ratio features** are those developed in the first pass to compare between alternatives.  These should be quite useful for random forests, because they condense the number of nodes needed to express the comparison of the cost features.  However, they might not be useful for logistic regression.  We can experiment.

Finally, there are 5 non-ordinal **encoded features** that we'll probably omit until we want much higher complexity.

In [9]:
simple_feats = ['SURVEY','LUGGAGE','INCOME','AGE','GA','SM_SEATS','CAR_AV','FIRST','MALE']
pure_feats = ['TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS', 'CAR_TT','CAR_CO']
encode_feats = ['PURPOSE','TICKET','WHO','ORIGIN','DEST']
ratio_feats = ['min_CO', 'ratio_TRAIN_CO','ratio_SM_CO', 'ratio_CAR_CO', 'min_TT', 'ratio_TRAIN_TT',
               'ratio_SM_TT','ratio_CAR_TT']


## Multivariate Logistic Regression
Or, "Multinomial Logit"

In [10]:
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegressionCV

In [11]:
X = df_e.drop("CHOICE", axis = 1)
y = df_e['CHOICE']

The split was chosen based on session-split in q_00.

In [12]:
split = 7002

In [13]:
Xtrain, Xtest, ytrain, ytest = ml_ut.tt_split(X,y,split)

To start, we'll take the pure numeric feats and the simple ordinal encodings or binary features

In [14]:
ML_feat = pure_feats + simple_feats

### Training

In [15]:
clf = LogisticRegression(multi_class='ovr').fit(Xtrain[ML_feat], ytrain)

In [16]:
clf2 = LogisticRegression(solver = "lbfgs", multi_class='multinomial').fit(Xtrain[ML_feat], ytrain)

In [17]:
clf_cv = LogisticRegressionCV(multi_class='ovr').fit(Xtrain[ML_feat], ytrain)
clf_cv2 = LogisticRegressionCV(multi_class='multinomial').fit(Xtrain[ML_feat], ytrain)

The Attribute "C\_" gives the value of the regularizer that is best for each class.  Because the 'refit' parameter is default true for this model, the whole model is refit on all of the training data after finding the best hyperparameter C for each class.

In [18]:
clf_cv.C_

array([  2.15443469e+01,   1.00000000e-04,   1.00000000e-04])

I think this is due to it being the multinomial case and learning the same parameters for the whole set.

In [19]:
clf_cv2.C_

array([ 0.35938137,  0.35938137,  0.35938137])

### Testing
We can predict on the whole set and then evaluate on the test and train separately.

In [20]:
py_x = clf.predict_proba(X[ML_feat])
dy_x = clf.predict(X[ML_feat])

Training

In [27]:
ml_ut.print_predict(X[ML_feat], clf, ytrain, ytest, split)

Training Log Loss: 0.699283805597
Training Accuracy: 0.69165952585
Testing  Log Loss: 0.784816724333
Testing  Accuracy: 0.585687382298


The accuracy is substantially higher on the training set, which is somewhat surprising. Logistic Regression is not something that I think usually overfits.  Let's look at the other models.

In [29]:
ml_ut.print_predict(X[ML_feat], clf2, ytrain, ytest, split)

Training Log Loss: 0.815291533847
Training Accuracy: 0.651242502142
Testing  Log Loss: 0.962744442308
Testing  Accuracy: 0.478611783697


The accuracy for the multinomial fit was worse than the one-versus-all method for the non-cross-validated case.

#### With Cross-Validation

In [30]:
ml_ut.print_predict(X[ML_feat], clf_cv, ytrain, ytest, split)

Training Log Loss: 0.787363181432
Training Accuracy: 0.657812053699
Testing  Log Loss: 0.942837909585
Testing  Accuracy: 0.478880817864


In [32]:
ml_ut.print_predict(X[ML_feat], clf_cv2, ytrain, ytest, split)

Training Log Loss: 0.806953372192
Training Accuracy: 0.651242502142
Testing  Log Loss: 0.953894843485
Testing  Accuracy: 0.477804681195


Surprisingly, we do substantially better without the cross-validation.  Perhaps there is another setting that I am overlooking. I did see that the best C values were at the bounds of the search space, which could mean that the bounds need to be widened.

#### Increase bounds of C parameter
The best performing was the dead-simplest.  Let's try to increase the bounds on the C parameter.

In [33]:
np.logspace(-1,2,4)

array([   0.1,    1. ,   10. ,  100. ])

In [34]:
model = LogisticRegressionCV(Cs = np.logspace(-10,2,13)).fit(Xtrain[ML_feat], ytrain)

In [35]:
ml_ut.print_predict(X[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.802159414805
Training Accuracy: 0.659097400743
Testing  Log Loss: 1.00768252553
Testing  Accuracy: 0.474307237019


In [36]:
model.C_

array([  1.00000000e+01,   1.00000000e-06,   1.00000000e-10])

In [38]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)
ml_ut.print_predict(X[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.699283805597
Training Accuracy: 0.69165952585
Testing  Log Loss: 0.784816724333
Testing  Accuracy: 0.585687382298


In [39]:
model.C

1.0

### Issue of C parameter
The regularization parameter is the only real parameter to be tuned for logistic regression, besides possibly the issue of one-vs-all against multinomial.  It is possible that the first models learned with cross validation are **worse with less data**, and so they learn sub-optimal parameters.  That is the only explanation I can think of.  We can move on to the inclusion or exclusion of feature sets.

## Grouping within Cross-validation

There is a larger possible issue of structure within the data, meaning that the order of samples in the data set is actually informative to the result.  If there is such an order, the cross validation will be hurt dramatically.  We could keep the train-test split for the sake of testing robustness to this, but the cross-validation grouping does not need to have this.  The input and ouput vectors must be shuffled together.

In [48]:
Xtrain = copy.deepcopy(X[:split])

In [49]:
shuffle_index = np.random.permutation(Xtrain.index)

In [50]:
Xtrain = Xtrain.reindex(shuffle_index)
ytrain = ytrain.reindex(shuffle_index)

Now the groups should at least not be stuck when doing the cross-validation.

Quick check here to confirm that the training is working as well as before.

In [53]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)

In [52]:
def print_predict2(X_ml_train, X_ml_test, model, ytrain, ytest, split):
    py_x_tr = model.predict_proba(X_ml_train)
    dy_x_tr = model.predict(X_ml_train)
    py_x_te = model.predict_proba(X_ml_test)
    dy_x_te = model.predict(X_ml_test)
    print("Training Log Loss: " + str(sklearn.metrics.log_loss(ytrain, py_x_tr)))
    print("Training Accuracy: " + str(sklearn.metrics.accuracy_score(ytrain, dy_x_tr)))
    print("Testing  Log Loss: " + str(sklearn.metrics.log_loss(ytest, py_x_te)))
    print("Testing  Accuracy: " + str(sklearn.metrics.accuracy_score(ytest, dy_x_te)))
    

In [55]:
print_predict2(Xtrain[ML_feat], Xtest[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.699158354787
Training Accuracy: 0.691373893173
Testing  Log Loss: 0.784915488786
Testing  Accuracy: 0.584880279796


#### Now try with cross-validation

In [56]:
model = LogisticRegressionCV().fit(Xtrain[ML_feat], ytrain)

In [57]:
print_predict2(Xtrain[ML_feat], Xtest[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.722908215667
Training Accuracy: 0.676092544987
Testing  Log Loss: 0.822443493302
Testing  Accuracy: 0.540758676352


Still worse!

In [58]:
model.C_

array([  1.00000000e+04,   2.78255940e+00,   2.15443469e+01])

Perhaps multinomial would improve it

In [59]:
model = LogisticRegressionCV(multi_class='multinomial').fit(Xtrain[ML_feat], ytrain)

In [60]:
print_predict2(Xtrain[ML_feat], Xtest[ML_feat], model, ytrain, ytest, split)

Training Log Loss: 0.803006862125
Training Accuracy: 0.652670665524
Testing  Log Loss: 0.949665233145
Testing  Accuracy: 0.476459510358


No better than before.

In [61]:
model.C_

array([ 21.5443469,  21.5443469,  21.5443469])

It does arrive at a parameter estimate 

In [34]:
model = LogisticRegressionCV(Cs = np.logspace(-10,2,13)).fit(Xtrain[ML_feat], ytrain)

## Feature Sets

In [3]:
ML_feat = pure_feats

NameError: name 'pure_feats' is not defined

In [ ]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)
print_predict(X[ML_feat], model, ytrain, ytest, split)

Worse than before - does better with more data.

In [ ]:
ML_feat = pure_feats + simple_feats + ratio_feats

In [ ]:
model = LogisticRegression().fit(Xtrain[ML_feat], ytrain)
print_predict(X[ML_feat], model, ytrain, ytest, split)

We **do gain information** from the ratio feats, which is encouraging.  

In [ ]:
# Accuracy scores from the simple random forest models
# log loss: 0.702412421415
# discrete accuracy: 0.684422921711

The accuracy still has some way to go, but now that we know that the logistic regression at least gives some meaningful result, we can investigate the relative strength of features.  

# Description of Remaining Features
Let's give a prose description of all the features that haven't yet been referenced.  **SURVEY** is the type of trip, train or car, for which the survey was given.  **ID** is the ID of the user - this is the session ID.  There are 1192 different sessions. Every row with the same ID has the same **CHOICE** value.  **PURPOSE** (9 categorical) was the purpose of the trip for which the survey was taken. **TICKET** (11 categorical) is the kind of ticket, round-trip, one-way, half-day, seasonal, etc.  **WHO** (4 categorical) is who pays: unknown, self, employer, half-half.  **LUGGAGE** (3 cat) 0 pieces, 1 piece, 3 several pieces.  **AGE** (6 cat) is an ordinal encoding of age, with greatest val unknown.  **INCOME** (5 cat) ordinal encoding of income level. **GA** - binary for owning an annual ticket. **ORIG** and **DEST** are 26-level cats encoding the region of travel.  They are probably too large to make useful, so while informative, they are unwieldy and could be dropped.  Then we have 3 types of straight numerical features - **TT** = Travel Time, in minutes, for each type, **CO** = cost, and **HE** = Headway or period between trains.  Cost for the car is computed as a fixed cost per unit distance.  Cost for the Train is computed based on the actual fare for an individual trip, or as the cost of the whole year with a GA - this makes the data quite hard to use, because there is a very clear bimodality in the distribution of costs.  The information it encodes isn't actually useful.  We would have to count on different cost thresholds being learnt for the GA/no GA cases.  Finally, **SM_SEATS** is a binary for the kinds of seats used on the Swiss Metro - airline style, or not.

The numerical features we probably want to keep, and we can hypothesize that they will be quite useful in choice prediction.  The origin and destination features probably have to be dropped.  Or, only keep the most frequent 5 or so values.  Purpose and Ticket might be useful, but are still fairly large for a one-hot encoding given how many samples there are.  The ordinal categorical features could be useful, but often have codings for 'unknown' that are at an extreme and so would hurt any linear model.  They could be treated as NAs and then just be sampled from the known distribution.  However, it is probably necessary to A/B test these kinds of data-filling changes.

#### Multiple Levels of Data
We could make several data sets: one with all of the features except for the truly unnecessary or redundant, and another, made beforehand for simplicity without the less likely features.  

In [ ]:
df1.to_csv('SM_clean.csv', index = False)

After transformations, we can make another saved set.

# The Target
On the CHOICE value.  This probably makes sense to keep as a multioutput problem.  For every row for each ID, there is a different set of Travel Time, Costs, Headway, and other parameters that determine the desirability of each method.  

It could however be transformed into a single output problem with session-level accuracy.  So each row could be converted into a set of different alternatives, each with their own travel-related parameters, but with the same customer parameters.  The target would be a binary chosen/not chosen feature.  Then, the output of the model would be a **utility**, rather than a probability, to be **normalized** to get the fractional 'shares allocation' of this user into different alternatives.  This is somewhat complicated by the fact that the different modes aren't easily comparable.  The car mode doesn't have a headway feature, and only SM has different seat configurations.  This generates about 3x the data, but a multioutput problem naturally creates models for each output.  

**Different Style**
It may be useful to train two different models: one to predict car v. not car, all car-user data goes here, then train v. SM, to which all train-only and car but train/SM choice data goes.

We have 1192 different individuals labelling their preferences, at 10728 total combinations of alternatives presented.

In [ ]:
len(set(df1['ID']))

In [ ]:
len(df1)

### Feature Sets

Here are some possible appropriate machine learning feature sets.

In [ ]:
ml_feat = ['SURVEY','FIRST','LUGGAGE','AGE','MALE','INCOME','GA','CAR_AV','TRAIN_TT',
          'TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','SM_SEATS','CAR_TT','CAR_CO']

In [ ]:
ml_feat = ['SURVEY','AGE','INCOME','GA','CAR_AV','TRAIN_TT',
          'TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','CAR_TT','CAR_CO']

In [ ]:
ml_feat = ['CAR_AV','TRAIN_TT','TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','CAR_TT','CAR_CO']

### Eliminate Unknown Choice values

There is little point in including a class with nine samples in the model, we could add it back in later if strictly necessary for comparison between models.

In [ ]:
df_k = copy.deepcopy(df1)

In [ ]:
df_k = df_k[df_k['CHOICE']!=0]

The eliminates nine rows.

In [ ]:
len(df_k)

### Valid Minimum Cost
We want to be able to create ratio features for the costs, which might improve learning by making direct comparisons.  For all rows, there is a train and SM cost, but there is only a car cost if the user has a car.  So to calculate a valid minimum for ratio features, we must account for this.

In [ ]:
def calc_valid_min(df):
    x = df1.loc[:,['SM_CO','TRAIN_CO','CAR_CO']].min(axis=1)
    for i in range(len(x)):
        if x[i]==0:
            x[i] = min(df1.loc[i,['SM_CO', 'TRAIN_CO']])
    return x            

In [ ]:
df_k.loc[:,'min_CO'] = calc_valid_min(df_k)

In [ ]:
ratio_price_feats = ['ratio_TRAIN_CO','ratio_SM_CO','ratio_CAR_CO']

In [ ]:
df_k.loc[:,'ratio_TRAIN_CO'] = df_k['TRAIN_CO']/df_k['min_CO']

df_k.loc[:,'ratio_SM_CO'] = df_k['SM_CO']/df_k['min_CO']
df_k.loc[:,'ratio_CAR_CO'] = df_k['CAR_CO']/df_k['min_CO']

### Valid Minimum Travel Time
Similarly, we want to create a set of travel time ratio features so the comparison can be done directly.

In [ ]:
def calc_valid_min2(df):
    x = df1.loc[:,['SM_TT','TRAIN_TT','CAR_TT']].min(axis=1)
    for i in range(len(x)):
        if x[i]==0:
            x[i] = min(df1.loc[i,['SM_TT', 'TRAIN_TT']])
    return x            

In [ ]:
df_k.loc[:,'min_TT'] = calc_valid_min2(df_k)

In [ ]:
df_k.loc[:,'ratio_TRAIN_TT'] = df_k['TRAIN_TT']/df_k['min_TT']
df_k.loc[:,'ratio_SM_TT'] = df_k['SM_TT']/df_k['min_TT']
df_k.loc[:,'ratio_CAR_TT'] = df_k['CAR_TT']/df_k['min_TT']

In [ ]:
ratio_feats = ratio_price_feats + ['ratio_TRAIN_TT','ratio_SM_TT','ratio_CAR_TT']

In [ ]:
ml_feat = ['SURVEY','FIRST','LUGGAGE','AGE','MALE','INCOME','GA','CAR_AV','TRAIN_TT',
          'TRAIN_CO','TRAIN_HE','SM_TT','SM_CO','SM_HE','SM_SEATS','CAR_TT','CAR_CO'] + ratio_feats
          

### Save the result
We'll save these likely transformations.

In [ ]:
df_k.to_csv('SM_expand.csv', index = False)